In [2]:
import pandas as pd

In [11]:
# 등록된 제품 수 by category, date
read_path = '../csv/products.csv'
df_product = pd.read_csv(read_path)

df_product.tail()


,date_at,category_id,status,product_num
1862041,2017-04-29,310050030.0,1,1
1862042,2019-02-01,500114005.0,2,1
1862043,2017-10-28,410200999.0,3,1
1862044,2017-05-19,900310100.0,3,1
1862045,2019-05-27,400083700.0,2,1


In [13]:
# 키워드 검색 수 by date
read_path = '../csv/keywords_1901_1904.csv'
df_keyword_1901_1904 = pd.read_csv(read_path)

read_path = '../csv/keywords_1807_1812.csv'
df_keyword_1807_1812 = pd.read_csv(read_path)

df_keyword = pd.concat([df_keyword_1901_1904, df_keyword_1807_1812])

# save
save_path = '../csv/tmp/df_keyword.csv'
df_keyword.to_csv(save_path, index=False, mode='w', header=True)

df_keyword.tail()

,date_at,keyword,search_num
81942714,2018-12-31,ìœ„ì¸ì „,1
81942715,2018-12-31,실리콘케이스 스카이블루,1
81942716,2018-12-31,오프화이트 갤럭시 후드집업,1
81942717,2018-12-31,후방카메라cctv,2
81942718,2018-12-31,패딩오버트라우져,1
